This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [1]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50
import numpy as np

In [2]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\train_dir"
test_dir  = r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\test_dir"

In [3]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [4]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 948 images belonging to 2 classes.


Found 236 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [5]:
initial_learning_rate = 0.001

# We are combining ExponentialDecay with Adam optimizer
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Load pretrained ResNet50 (no top layers)
base_model = ResNet50(
    weights='imagenet',
    include_top=False,
    input_shape=(128, 128, 3)
)

# Freeze pretrained weights
base_model.trainable = False

# Build new model with better regularization
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 14s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ resnet50 (Functional)           │ (None, 4, 4, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           257 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,112,513 (91.98 MB)

 Trainable params: 524,801 (2.00 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# early stop if validation stops improving
early_stop = EarlyStopping(
    monitor='val_loss', 
    patience=3,  # wait 3 epochs before stopping
    restore_best_weights=True
)

# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stop]
)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 216s 5s/step - accuracy: 0.5401 - loss: 0.9373 - val_accuracy: 0.6186 - val_loss: 0.7186
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 69s 2s/step - accuracy: 0.5633 - loss: 0.7146 - val_accuracy: 0.7076 - val_loss: 0.6529
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.6508 - loss: 0.6569 - val_accuracy: 0.6441 - val_loss: 0.6183
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 64s 2s/step - accuracy: 0.6688 - loss: 0.6283 - val_accuracy: 0.5975 - val_loss: 0.6616
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.6572 - loss: 0.6409 - val_accuracy: 0.6780 - val_loss: 0.6265
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 51s 2s/step - accuracy: 0.6835 - loss: 0.6279 - val_accuracy: 0.7585 - val_loss: 0.5708
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 49s 2s/step - accuracy: 0.7152 - loss: 0.6052 - val_accuracy: 0.8178 - val_loss: 0.5445
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 66s 2s/step - accuracy: 0.7300 - loss: 0.5846 - val_accuracy: 0.7966 - val_loss

In [8]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 113s 3s/step - accuracy: 0.7669 - loss: 0.5453
Test Accuracy: 0.7668918967247009


In [9]:
# SAVE THE MODEL
model.save('muffin_vs_chihuahua_cnn_resnet.h5')

In [10]:
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='muffin_vs_chihuahua_cnn_resnet.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [12]:
# Example usage:
predict_image(r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\run\run_1.jpg")
predict_image(r"C:\Users\USER\25-26\CS 3B\JETHRO_RENDON\run\run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Muffin (confidence: 0.53)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Prediction: Chihuahua (confidence: 0.33)
